In [ ]:
import os
import time
import json
import cv2
from skimage.io import imsave
import torch
from maskrcnn_benchmark.config import cfg
from predictor import COCODemo
import numpy as np
from PIL import Image
from maskrcnn_benchmark.layers import nms as _box_nms

if not os.path.exists('result'):
    os.mkdir('result')
    
if not os.path.exists('result_resize'):
    os.mkdir('result_resize')
    
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def load_image_into_numpy_array_resize(image,k):
    (im_width, im_height) = image.size
    im_width = int(im_width*k)
    im_height = int(im_height*k)
    
    image = image.resize((im_width, im_height), Image.ANTIALIAS)
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def predict_save(coco_demo, test_img_fold, test_img_list):
    # load image
    sub_dict = {}
    sub_dict['results'] = []
    #for i in range(2):
    for i in range(len(test_img_list)):
        # process image
        start = time.time()
        # for i in range(1):
        img_name = test_img_list[i]
        img_path = os.path.join(test_img_fold, img_name)

        result = predict_resize_img(coco_demo,img_path)
    
        boxes = result['boxes']
        scores = result['scores']
        labels = result['labels']
        print("processing time: ", time.time() - start)
        # correct for image scale
        
        image_dict = {}
        image_dict['filename'] = img_name
        image_dict['rects'] = []
        
        for box, score, label in zip(boxes, scores, labels):
            # scores are sorted so we can break
            print (box, score, label)
            if score < 0.5:
                break
            box_dict = {}
            color = 'red' #label_color(label)
            box_dict['xmin'] = int(box[0])
            box_dict['ymin'] = int(box[1])
            box_dict['xmax'] = int(box[2])
            box_dict['ymax'] = int(box[3])
            box_dict['label'] = int(label)
            box_dict["confidence"] = float(score)
            image_dict['rects'].append(box_dict)
            
        sub_dict['results'].append(image_dict)
    with open("record.json","w") as f:
        json.dump(sub_dict,f)
        print("加载入文件完成...")


In [ ]:
FLIP_LEFT_RIGHT = 0
FLIP_TOP_BOTTOM = 1

import cv2
import matplotlib.pyplot as plt
%matplotlib inline
color_dict = {1:(255,0,0),2:(0,255,0),3:(0,0,255),4:(255,255,0),5:(0,255,255)}
def overlay_boxes(image, boxes,labels):
    """
    Adds the predicted boxes on top of the image

    Arguments:
        image (np.ndarray): an image as returned by OpenCV
        predictions (BoxList): the result of the computation by the model.
            It should contain the field `labels`.
    """
#     labels = predictions.get_field("labels")
#     boxes = predictions.bbox

    colors = [color_dict[int(label)] for label in labels]

    for box, color in zip(boxes, colors):
        box = box.to(torch.int64)
        top_left, bottom_right = box[:2].tolist(), box[2:].tolist()
        image = cv2.rectangle(
            image, tuple(top_left), tuple(bottom_right), tuple(color), 1
        )

    return image

def predict_single_img(coco_demo,img_path):
    image = Image.open(img_path)
    # the array based representation of the image will be used later in order to prepare the
    image_np = load_image_into_numpy_array(image)  
    image_np,predictions = coco_demo.run_on_opencv_image(image_np)
    # print(image.shape)
    # print(scale)
    boxes = predictions.bbox
    scores = predictions.get_field("scores")
    labels = predictions.get_field("labels")
    result['boxes'] = torch.index_select(box_cat,0,keep)
    result['scores'] = torch.index_select(score_cat,0,keep)
    result['labels'] = torch.index_select(label_cat,0,keep)
    print (result)
    result['size'] = image.size
    imsave('result/'+img_path.split('/')[-1], image_np)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.show()
    return result

def predict_resize_img(coco_demo,img_path):
    boxlist = []
    scorelist = []
    labellist = []
    img_name = img_path.split('/')[-1].split('.')
    for k in [0.5,1,2]:#[0.5,0.75,1,1.5,2]:
        image = Image.open(img_path)
        (im_width, im_height) = image.size
        # the array based representation of the image will be used later in order to prepare the
        image_np = load_image_into_numpy_array_resize(image,k)  
        re,predictions = coco_demo.run_on_opencv_image(image_np)
        #imsave('result_resize/'+img_name[0]+'_'+str(k)+'.'+img_name[1], re)
        boxes = predictions.resize((im_width, im_height)).bbox
        scores = predictions.get_field("scores")
        labels = predictions.get_field("labels")
        
        boxlist.append(boxes)
        scorelist.append(scores)
        labellist.append(labels)
    
    for i in range(3):
        image = Image.open(img_path)
        if i == 1:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
        elif i == 2:
            image = image.transpose(Image.FLIP_TOP_BOTTOM)
        (im_width, im_height) = image.size
        k = 800./min(im_width, im_height)
        image_np = load_image_into_numpy_array_resize(image,k)
        re,predictions = coco_demo.run_on_opencv_image(image_np)
        if i==0:
            #imsave('result_resize/'+img_name[0]+'_normal.'+img_name[1], re)
            boxes = predictions.resize((im_width, im_height)).bbox
        elif i ==1:
            #imsave('result_resize/'+img_name[0]+'_lr.'+img_name[1], re)
            boxes = predictions.transpose(FLIP_LEFT_RIGHT).resize((im_width, im_height)).bbox
        elif i ==2:
            #imsave('result_resize/'+img_name[0]+'_tb.'+img_name[1], re)
            boxes = predictions.transpose(FLIP_TOP_BOTTOM).resize((im_width, im_height)).bbox
        scores = predictions.get_field("scores")
        labels = predictions.get_field("labels")
        boxlist.append(boxes)
        scorelist.append(scores)
        labellist.append(labels)
        
    box_cat = torch.cat(boxlist,0)
    score_cat = torch.cat(scorelist,0)
    label_cat = torch.cat(labellist,0)
    image = Image.open(img_path)
    nms_thresh = 0.5
    keep = _box_nms(box_cat, score_cat, nms_thresh)
    result = {}
    scores = torch.index_select(score_cat,0,keep)
    boxes = torch.index_select(box_cat,0,keep)
    labels = torch.index_select(label_cat,0,keep)
    result['boxes'] = boxes[scores>0.5]
    result['scores'] = scores[scores>0.5]
    result['labels'] = labels[scores>0.5]
    #print (result)
    result['size'] = image.size
    image_np=load_image_into_numpy_array(image)
    image_np = overlay_boxes(image_np, result['boxes'],result['labels'])
    imsave('result/'+img_path.split('/')[-1], image_np)
    #plt.figure(figsize=(12, 8))
    #plt.imshow(image_np)
    #plt.show()
    return result
    

In [ ]:
config_file = "../configs/e2e_faster_rcnn_X_101_32x8d_FPN_1x_val.yaml"
# update the config options with the config file
cfg.merge_from_file(config_file)
# manual override some options
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])

coco_demo = COCODemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.7,
)



In [ ]:
predict_save(coco_demo, test_img_fold, test_img_list)

In [ ]:
# print(model.summary())
#
#labels_to_names = {1: 't', 2: 'h', 3: 'd', 4: 'd', 5: 'j'}
# load label to names mapping for visualization purposes
labels_to_names = {0: 'tieke', 1: 'heiding',
                   2: 'daoju', 3: 'dian', 4: 'jiandao'}

test_img_fold = '/home/hywel/Documents/maskrcnn-benchmark/datasets/coco/rest'
#test_img_fold = '/home/hywel/Documents/keras-retinanet/keras_retinanet/CSV/jinnan2_round1_test_a_20190306'
test_img_list = os.listdir(test_img_fold)
print(len(test_img_list))

In [ ]:
train_annotations_file = '/home/hywel/Documents/maskrcnn-benchmark/datasets/coco/annotations/train_argument.json'
def xywh_xyxy(box):
    box2 = []
    for i in range(len(box)):
        box2.append( [box[i][0],box[i][1],box[i][0]+box[i][2],box[i][1]+box[i][3]] )
    
    return box2
def find_gt(image_name,img_fold,annotations_file):
    with open(annotations_file,'r') as load_f:
        groundtruth_data = json.load(load_f)
    images = groundtruth_data['images']
    annotations = groundtruth_data['annotations']
    image_data = None
    for image in images:
        if image_name == image['file_name']:
            image_data = image
            break
    if not image_data: 
        print ('can not find image')
        return
    boxes = []
    labels = []
    minAreaRects = []
    for annotation in annotations:
        if annotation['image_id'] == image_data['id']:
            boxes.append(annotation['bbox'])
            labels.append(annotation['category_id'])
            minAreaRects.append(annotation['minAreaRect'])
            
    result={}
    result['boxes'] = torch.tensor(xywh_xyxy(boxes))
    result['minAreaRects'] = minAreaRects
    result['labels'] = torch.tensor(labels)
    #print (result)
    result['size'] = (image_data["width"],image_data["height"])
    print (image_name)
    image = Image.open(img_fold+'/'+image_name)
    image_np=load_image_into_numpy_array(image)
    image_np = overlay_boxes(image_np, result['boxes'],result['labels'])
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.show()
    return result


In [ ]:
for i in range(50):
    find_gt(test_img_list[i],test_img_fold,train_annotations_file)

In [ ]:
test_img_list[1]

In [ ]:
#增强对比度
def contrast_brightness_image(src1, a, g):
    h, w, ch = src1.shape#获取shape的数值，height和width、通道
 
    #新建全零图片数组src2,将height和width，类型设置为原图片的通道类型(色素全为零，输出为全黑图片)
    src2 = np.zeros([h, w, ch], src1.dtype)
    dst = cv2.addWeighted(src1, a, src2, 1-a, g)#addWeighted函数说明如下
    plt.figure(figsize=(12, 8))
    plt.imshow(dst[..., -1::-1])
    plt.show()

def equalize_clahe_image(image_path):
    print (image_path)
    img = cv2.imread(image_path)#.convert('RGB')
    plt.figure(figsize=(12, 8))
    plt.imshow(img[..., -1::-1])
    plt.show()
    contrast_brightness_image(img, 1.2, 10)
    
equalize_clahe_image(test_img_fold+'/'+test_img_list[1])